# TF-IDF로 단어 벡터화, k-fold로 교차검증

In [3]:
import pandas as pd
"""
header = 0 은 파일의 첫 번째 줄에 열 이름이 있음을 나타내며 
delimiter = \t 는 필드가 탭으로 구분되는 것을 의미한다.
quoting = 3은 쌍따옴표를 무시하도록 한다.
"""
# QUOTE_MINIMAL (0), QUOTE_ALL (1), 
# QUOTE_NONNUMERIC (2) or QUOTE_NONE (3).

# 레이블인 sentiment 가 있는 학습 데이터
train = pd.read_csv('word_tutorial/labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)
# 레이블이 없는 테스트 데이터
test = pd.read_csv('word_tutorial/testData.tsv', header=0, delimiter='\t', quoting=3)
print(train.shape)
print(test.shape)

(25000, 3)
(25000, 2)


In [4]:
train['sentiment'].value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [5]:
# 데이터 전처리는 태그만 제거하는 정도로
# WordNetLemmatizer로 레마타이징한다.
from KaggleWord2VecUtility import KaggleWord2VecUtility
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

def review_to_words(raw_review):
    review_text = BeautifulSoup(raw_review, 'lxml').get_text()
    review_text = wordnet_lemmatizer.lemmatize(review_text)
    return review_text

In [6]:
# 학습 데이터 전처리 / worker 나눠서 하는게 현재 안 된다
# %time train['review'] = KaggleWord2VecUtility.apply_by_multiprocessing(train['review'], review_to_words, workers=4)

clean_train_reviews = []
for i in range(0, train['review'].size):
    clean_train_reviews.append(review_to_words(train['review'][i]))

train['review_clean'] = clean_train_reviews
train['review_clean'].head()

0    "With all this stuff going down at the moment ...
1    "\"The Classic War of the Worlds\" by Timothy ...
2    "The film starts with a manager (Nicholas Bell...
3    "It must be assumed that those who praised thi...
4    "Superbly trashy and wondrously unpretentious ...
Name: review_clean, dtype: object

In [7]:
# 테스트 데이터 전처리
# %time test['review'] = KaggleWord2VecUtility.apply_by_multiprocessing(test['review'], review_to_words, workers=4)

clean_test_reviews = []
for i in range(0, test['review'].size):
    clean_test_reviews.append(review_to_words(test['review'][i]))

test['review_clean'] = clean_test_reviews
test['review_clean'].head()

0    "Naturally in a film who's main themes are of ...
1    "This movie is a disaster within a disaster fi...
2    "All in all, this is a movie for kids. We saw ...
3    "Afraid of the Dark left me with the impressio...
4    "A very accurate depiction of small time mob l...
Name: review_clean, dtype: object

In [8]:
# X_train과 X_test에 리뷰 데이터 담고 이 데이터를 TF-IDF 통해 임베딩(벡터화)해본다
X_train = train['review_clean']
X_test = test['review_clean']

In [14]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

## TF-IDF

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from nltk.corpus import words

vectorizer = CountVectorizer(analyzer = 'word', 
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3),
                             vocabulary = set(words.words()), # nltk의 words를 사용하거나 문서 자체의 사전을 만들거나 선택한다. 
                             max_features = 90000
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=90000, min_df=2,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None,
        vocabulary={'Humiriaceous', 'xerotherm', 'centrifugation', 'exposedness', 'protobasidiomycetous', 'concordal', 'pennyleaf', 'uncomplaisance', 'unexceptable', 'piny', 'perpendicular', 'alphol', 'interrelated', 'horsegate', 'venefical', 'rondawel', 'eurygnathous', 'baldberry', 'polygyria', 'soldering'... 'uncanvassed', 'mijl', 'theogonism', 'yotacism', 'confidentiary', 'routinely', 'irresonant', 'hox'})

## TfidfTransformer()
- norm='l2' 각 문서의 피처 벡터를 어떻게 벡터 정규화 할지 정한다
    - L2 : 벡터의 각 원소의 제곱의 합이 1이 되도록 만드는 것이고 기본 값
    - L1 : 벡터의 각 원소의 절댓값의 합이 1이 되도록 크기를 조절
- smooth_idf=False
    - 피처를 만들 때 0으로 나오는 항목에 대해 작은 값을 더해서(스무딩을 해서) 피처를 만들지 아니면 그냥 생성할지를 결정
- sublinear_tf=False
- use_idf=True
    - TF-IDF를 사용해 피처를 만들 것인지 아니면 단어 빈도 자체를 사용할 것인지 여부

In [10]:
pipeline = Pipeline([
    ('vect', vectorizer),
    ('tfidf', TfidfTransformer(smooth_idf=False)),
])
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=90000, min_df=2,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
       ...('tfidf', TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True))])

In [11]:
X_train_tfidf_vector = pipeline.fit_transform(X_train)

c:\python36\virtualenv\venv01\lib\site-packages\sklearn\feature_extraction\text.py:1247: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [12]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:5]

235892


['A', 'Aani', 'Aaron', 'Aaronic', 'Aaronical']

In [13]:
X_test_tfidf_vector = pipeline.fit_transform(X_test)

In [14]:
import numpy as np

dist = np.sum(X_train_tfidf_vector, axis=0)

for tag, count in zip(vocab, dist):
    print(count, tag)
    
pd.DataFrame(dist, columns=vocab)

[[0. 0. 0. ... 0. 0. 0.]] A


,A,Aani,Aaron,Aaronic,Aaronical,Aaronite,Aaronitic,Aaru,Ab,Ababdeh,...,zymotechnical,zymotechnics,zymotechny,zymotic,zymotically,zymotize,zymotoxic,zymurgy,zythem,zythum
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# 랜덤 포레스트 분류기 사용
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [14]:
%time forest = forest.fit(X_train_tfidf_vector, train['sentiment'])

Wall time: 23.9 s


## 교차검증 (k-fold 이용)

In [15]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=5, shuffle=True, random_state=2018)
%time score = np.mean(cross_val_score(forest, X_train_tfidf_vector, train['sentiment'], cv=k_fold, n_jobs=-1, scoring='roc_auc'))

Wall time: 1min 37s


In [16]:
score

0.920512825300707

In [17]:
%time result = forest.predict(X_test_tfidf_vector)

Wall time: 554 ms


In [18]:
result[:15]

array([1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0], dtype=int64)

In [19]:
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.tail()

,id,sentiment
24995,"""2155_10""",1
24996,"""59_10""",1
24997,"""2531_1""",1
24998,"""7772_8""",1
24999,"""11465_10""",1


In [20]:
output_sentiment = output['sentiment'].value_counts()
output_sentiment

1    12689
0    12311
Name: sentiment, dtype: int64

In [ ]:
# csv로 저장하기
# output.to_csv('word_tutorial/tutorial_4_tfidf_{0:.5f}.csv'.format(score), index=False, quoting=3)

# XGBoost : A Scalable Tree Boosting System
- 분산형 그래디언트 부스팅 알고리즘
- 부스팅 알고리즘

In [15]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train_tfidf_vector, label=train['sentiment'])

c:\python36\virtualenv\venv01\lib\site-packages\xgboost-0.81-py3.6.egg\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\python36\virtualenv\venv01\lib\site-packages\xgboost-0.81-py3.6.egg\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [16]:
dtrain

In [25]:
# 멀티프로세싱은 nthread 였는데 n_jobs로 변경되었다고 한다.
# 설치 된 xgboost버전에 따라 파라메터가 다를 수 있으니 
# 이 코드를 돌리고 터미널에서 $ top -o cpu 로 CPU자원을 100%넘게 사용하고 있는지 확인해 본다.
# 파리미터 정보 : https://xgboost.readthedocs.io/en/latest/parameter.html
'''
params = {
    'booster': 'gblinear',
    'objective': 'multi:softmax',
    'eval_metric': 'merror',
    'eta' : 0.02,
    'lambda': 2.0,
    'alpha': 1.0,
    'lambda_bias': 6.0,
    'num_class': 5,
    'n_jobs' : 4,
    'silent': 0,
}
'''
params = {
    # General parameter
    'booster': 'gbtree',   # gbliner도 있음
    'silent': 0,
    'n_jobs' : 4,   # 가동할 코어 개수
    
    # Booster parameter (gbtree booster 기준으로 정리)
    'eta' : 0.05,   # learning rate라고 생각, 일반적으로 0.01 ~ 0.2
    'max_depth' : 4,   # 트리 최대 깊이 (보통 3 ~ 10) / default=6
    'lambda': 1.5,   # default=1, 가중치에 대한 L2 정규화 용어 => reg_lambda (Ridge 회귀 분석과 유사)
    'alpha': 1.0,   # default=0, 가중치에 대한 L1 정규화 용어 => reg_alpha (Lasso 회귀 분석과 유사)  
    # max_leaf_nodes : 최종 노드의 최대 개수, 2^max_depth
    # min_child_weight [default=1] : 오버피팅, 언더피팅 조정, 너무 크면 오버피팅
    # gamma [default=0] : 분할 수행하는데 필요한 최소 손실 감소 지정
    # subsample [default=1] : 각 트리마다의 관측 데이터 샘플링 비율, 너무 작으면 언더피팅, 0.5 ~ 1.0
    # colsample_bytree [default=1] : 각 트리마다의 feature 샘플링 비율, 0.5 ~ 1.0
    # scale_pos_weight [default=1] : 불균형한 경우 더 빠른 수렴에 도움
  
    # learning Task parameter : 각 단계에서 계산할 최적화 목표 정의
    'objective': 'multi:softmax',   # default=reg:linear / binary:logistic, multi:softmax, multi:softprob 등등
    'num_class': 5,   # multi:softmax 에 필요
    'eval_metric': 'merror',   # default=rmse(회귀 분석) or error(클래스 분류) / rmse, mae, logloss, error, merror, mlogloss, auc 등등
    # seed [default=0] : 난수 시드
}

%time booster = xgb.train(params, dtrain, num_boost_round=500)

Wall time: 2min 31s


In [22]:
dtest = xgb.DMatrix(X_test_tfidf_vector)

result = booster.predict(dtest)

print(result.shape)
result[0:10]

(25000,)


array([1., 0., 1., 1., 1., 0., 0., 1., 0., 1.], dtype=float32)

In [23]:
result = result.astype(int)
output = pd.DataFrame(data={'id':test['id'], 'sentiment':result})
output.head(10)

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",1
4,"""12128_7""",1
5,"""2913_8""",0
6,"""4396_1""",0
7,"""395_2""",1
8,"""10616_1""",0
9,"""9074_9""",1


In [24]:
output_sentiment = output['sentiment'].value_counts()
output_sentiment

1    13958
0    11042
Name: sentiment, dtype: int64

# prediction 문제가 생긴다...
- 해결 : gblinear를 gbtree로 바꾸니 여러 예측값 나옴

In [28]:
print(dtrain)